# Mod 11 CS 1

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
from sklearn.metrics import mean_squared_error

In [2]:
df_book_rat = pd.read_csv("BX-Book-Ratings.csv", encoding = 'latin1', header=0)
df_book_rat.shape

(1048575, 3)

In [3]:
df_book_rat.head()

,user_id,isbn,rating
0,276725,034545104X,0
1,276726,155061224,5
2,276727,446520802,0
3,276729,052165615X,3
4,276729,521795028,6


In [4]:
#Unique Ratings
unique_ratings = set(df_book_rat["rating"])
unique_ratings

{0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10}

In [5]:
df_book_rat.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1048575 entries, 0 to 1048574
Data columns (total 3 columns):
 #   Column   Non-Null Count    Dtype 
---  ------   --------------    ----- 
 0   user_id  1048575 non-null  int64 
 1   isbn     1048575 non-null  object
 2   rating   1048575 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 24.0+ MB


In [6]:
df_book_rat.describe()

,user_id,rating
count,1.048575e+06,1.048575e+06
mean,1.285089e+05,2.879907e+00
std,7.421876e+04,3.857870e+00
min,2.000000e+00,0.000000e+00
25%,6.339400e+04,0.000000e+00
50%,1.288350e+05,0.000000e+00
75%,1.927790e+05,7.000000e+00
max,2.788540e+05,1.000000e+01


In [7]:
#Sorting the dataframe and extracting the first 10000 records

df_book_rat.sort_values(["user_id" ,"isbn"], inplace =True)
df_book_rat1 = df_book_rat.head(10000)
df_book_rat1.reset_index()
df_book_rat1.head()

,user_id,isbn,rating
9561,2,195153448,0
9562,7,34542252,0
9571,8,074322678X,5
9574,8,080652121X,0
9576,8,1552041778,5


In [8]:
#Preprocessing the isbn
from sklearn.preprocessing import LabelEncoder
lbl_enc = LabelEncoder()
df_book_rat1['isbn'] = lbl_enc.fit_transform(df_book_rat1['isbn'])
df_book_rat1.head()

/srv/conda/envs/notebook/lib/python3.6/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


,user_id,isbn,rating
9561,2,1467,0
9562,7,2490,0
9571,8,443,5
9574,8,475,0
9576,8,877,5


In [9]:
df_books = pd.read_csv("BX-Books.csv", encoding="latin1", low_memory=False)
df_books.head()

,isbn,book_title,book_author,year_of_publication,publisher
0,195153448,Classical Mythology,Mark P. O. Morford,2002,Oxford University Press
1,2005018,Clara Callan,Richard Bruce Wright,2001,HarperFlamingo Canada
2,60973129,Decision in Normandy,Carlo D'Este,1991,HarperPerennial
3,374157065,Flu: The Story of the Great Influenza Pandemic...,Gina Bari Kolata,1999,Farrar Straus Giroux
4,393045218,The Mummies of Urumchi,E. J. W. Barber,1999,W. W. Norton &amp; Company


In [10]:
df_users = pd.read_csv("BX-Users.csv", encoding="latin1", low_memory=False)
df_users.head()

,user_id,Location,Age
0,1,"nyc, new york, usa",NaN
1,2,"stockton, california, usa",18.0
2,3,"moscow, yukon territory, russia",NaN
3,4,"porto, v.n.gaia, portugal",17.0
4,5,"farnborough, hants, united kingdom",NaN


In [11]:
n_users = df_book_rat1["user_id"].unique().shape[0]
n_users

1323

In [12]:
n_books = df_book_rat1["isbn"].unique().shape[0]
n_books

8742

In [13]:
data_mat = np.zeros([n_users, n_books])
data_mat.shape

(1323, 8742)

In [14]:
for line in df_book_rat1.head().itertuples():
    data_mat[line[1]-1, line[2]-1] = line[3]

In [15]:
from sklearn.metrics.pairwise import pairwise_distances
user_similarity = pairwise_distances(data_mat, metric='cosine')
item_similarity = pairwise_distances(data_mat.T, metric='cosine')

In [16]:
def predict(ratings, similarity, type):
    if type == 'user':
        mean_user_rating = ratings.mean(axis=1)
        ratings_diff = (ratings - mean_user_rating[:, np.newaxis])
        
        pred = mean_user_rating[:, np.newaxis] + similarity.dot(ratings_diff) / np.array([np.abs(similarity).sum(axis=1)]).T
    
    elif type == 'book':
        pred = ratings.dot(similarity) / np.array([np.abs(similarity).sum(axis=1)])
    return pred

In [17]:
from sklearn.metrics.pairwise import paired_distances
user_prediction = predict(data_mat, user_similarity, type='user')
item_prediction = predict(data_mat, item_similarity, type='book')

In [20]:
# Root Mean square

def rmse(pred, test):
    pred =pred[test.nonzero()].flatten()
    test = test[test.nonzero()].flatten()
    return np.sqrt(mean_squared_error(pred, test))

In [21]:
rmse(user_prediction, data_mat)

4.9988560970029745

In [22]:
rmse(item_prediction, data_mat)

5.0